https://en.wikipedia.org/wiki/Gauss%E2%80%93Newton_algorithm

## toy problem, code up the algo myself

In [61]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as pt
import scipy.linalg as la
from scipy.optimize import least_squares
%matplotlib inline

In [29]:
a = 2
b = 0.5

In [30]:
underlying_function = lambda x: a*x**b

In [31]:
x = np.array([1,2,3,4])
y = [underlying_function(x_i) for x_i in x]

In [32]:
def residual(parameter):
    a, b = parameter
    return y - a * x**b

In [41]:
def jacobian(parameter):
    a, b = parameter
    return np.array([
        -x**b,
        -a*np.log(x)*x**b
        ]).T

In [53]:
p = np.array([5.0,1.0])
    

In [54]:
while True:
    J = jacobian(p)
    r = residual(p)
    step = la.inv(J.T@J)@J.T@r
    p -= step
    
    if np.abs(step).max() < 0.0001:
        break

# using scipy.optimize.least_squares

In [67]:
x0 = np.array([2, 2])
res_1 = least_squares(residual, x0)
res_1.x




array([2. , 0.5])

In [68]:
res_1.cost


7.784092131320288e-25

In [69]:
res_1.jac


array([[-1.        ,  0.        ],
       [-1.41421356, -1.96051627],
       [-1.7320508 , -3.80570465],
       [-2.00000001, -5.54517752]])

In [70]:
jacobian([2. , 0.5])

array([[-1.        , -0.        ],
       [-1.41421356, -1.96051629],
       [-1.73205081, -3.8057046 ],
       [-2.        , -5.54517744]])